# High Performance Spark Queries with Databricks Delta
Databricks Delta extends Apache Spark to simplify data reliability and boost Spark's performance.

Building robust, high performance data pipelines can be difficult due to: lack of indexing and statistics, data inconsistencies introduced by schema changes and pipeline failures, and having to trade off between batch and stream processing.

With Databricks Delta, data engineers can build reliable and fast data pipelines. Databricks Delta provides many benefits including:

Faster query execution with indexing, statistics, and auto-caching support
Data reliability with rich schema validation and transactional guarantees
Simplified data pipeline with flexible UPSERT support and unified Structured Streaming + batch processing on a single data source.
## Let's See How Databricks Delta Makes Spark Queries Faster!
In this example, we will see how Databricks Delta can optimize query performance. We create a standard table using Parquet format and run a quick query to observe its latency. We then run a second query over the Databricks Delta version of the same table to see the performance difference between standard tables versus Databricks Delta tables.

Simply follow these 4 steps below:

- Step 1 : Create a standard Parquet based table using Data 4 for CVS - `gs://cvs-synthetic/dipali-test/profile1` 3.67 GiB
- Step 2 : Run a query to to fetch a record by `MBR_ACCT_GID` which is the primary ID
- Step 3 : Create the cvs table using Databricks Delta and optimize the table on `MBR_ACCT_GID` since this is the column on which we want to run fetch and has highest cardinality.
- Step 4 : Rerun the query in Step 2 and observe the latency.
<br>
<b>Note</b>: Throughout the example we will be building few tables with a 10s of million rows. Some of the operations may take a few minutes depending on your cluster configuration.

In [ ]:
%fs ls /tmp/data

path,name,size,modificationTime
dbfs:/tmp/data/000000000000.parquet,000000000000.parquet,39723836,1663667225000
dbfs:/tmp/data/000000000001.parquet,000000000001.parquet,39356828,1663667239000
dbfs:/tmp/data/000000000002.parquet,000000000002.parquet,39362872,1663667253000
dbfs:/tmp/data/000000000003.parquet,000000000003.parquet,39421412,1663667232000
dbfs:/tmp/data/000000000004.parquet,000000000004.parquet,39415391,1663667237000
dbfs:/tmp/data/000000000005.parquet,000000000005.parquet,39435480,1663667268000
dbfs:/tmp/data/000000000006.parquet,000000000006.parquet,39418602,1663667244000
dbfs:/tmp/data/000000000007.parquet,000000000007.parquet,39241704,1663667242000
dbfs:/tmp/data/000000000008.parquet,000000000008.parquet,39577482,1663667255000
dbfs:/tmp/data/000000000009.parquet,000000000009.parquet,39152124,1663667227000


In [ ]:
%fs rm -r /tmp/cvs_parquet 

res1: Boolean = false

In [ ]:
%fs rm -r /tmp/cvs_delta

res2: Boolean = false

##### Set the Service Account json which has access to `gs://cvs-synthetic/dipali-test/profile2/` this bucket

In [ ]:
import json

service_account_json ={}
with open("sa.json", "w+") as outfile:
    json.dump(service_account_json, outfile)

##### Installing gcloud sdk to bring in data from GCS

In [ ]:
!curl -O https://dl.google.com/dl/cloudsdk/channels/rapid/downloads/google-cloud-cli-402.0.0-linux-x86.tar.gz
!tar -xf google-cloud-cli-402.0.0-linux-x86.tar.gz
!echo "N" | ./google-cloud-sdk/install.sh 
!./google-cloud-sdk/bin/gcloud auth activate-service-account --key-file sa.json

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0 31 90.0M 31 27.9M 0 0 30.5M 0 0:00:02 --:--:-- 0:00:02 30.5M 74 90.0M 74 66.9M 0 0 33.9M 0 0:00:02 0:00:01 0:00:01 33.9M100 90.0M 100 90.0M 0 0 31.7M 0 0:00:02 0:00:02 --:--:-- 31.7M
Welcome to the Google Cloud CLI!
WARNING: You appear to be running this script as root. This may cause 
the installation to be inaccessible to users other than the root user.

To help improve the quality of this product, we collect anonymized usage data
and anonymized stacktraces when crashes are encountered; additional information
is available at <https://cloud.google.com/sdk/usage-statistics>. This data is
handled in accordance with our privacy policy
<https://cloud.google.com/terms/cloud-privacy-notice>. You may choose to opt in this
collection now (by choosing 'Y' at the below prompt), or at any time in the
future by running the following command:

 gcloud config set disable_usage_reporting false

Do you want to help improve the Google Cloud CLI (y/N)? 

Your current Google Cloud CLI version is: 402.0.0
The latest available version is: 408.0.1

┌───────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Components │
├──────────────────┬──────────────────────────────────────────────────────┬──────────────────────────┬──────────┤
│ Status │ Name │ ID │ Size │
├──────────────────┼──────────────────────────────────────────────────────┼──────────────────────────┼──────────┤
│ Update Available │ BigQuery Command Line Tool │ bq │ 1.6 MiB │
│ Update Available │ Cloud Storage Command Line Tool │ gsutil │ 15.5 MiB │
│ Update Available │ Google Cloud CLI Core Libraries │ core │ 25.1 MiB │
│ Not Installed │ App Engine Go Extensions │ app-engine-go │ 4.2 MiB │
│ Not Installed │ Appctl │ appctl │ 21.0 MiB │
│ Not Installed │ Artifact Registry Go Module Package Helper │ package-go-module │ < 1 MiB │
│ Not Installed │ Bundled Python 3.9 │ bundled-python3-unix │ 62.2 MiB │
│ Not Installed │ Cloud Bigtable Command Line Tool │ cbt │ 10.2 MiB │
│ Not Installed │ Cloud Bigtable Emulator │ bigtable │ 6.7 MiB │
│ Not Installed │ Cloud Datalab Command Line Tool │ datalab │ < 1 MiB │
│ Not Installed │ Cloud Datastore Emulator │ cloud-datastore-emulator │ 35.1 MiB │
│ Not Installed │ Cloud Firestore Emulator │ cloud-firestore-emulator │ 40.2 MiB │
│ Not Installed │ Cloud Pub/Sub Emulator │ pubsub-emulator │ 62.4 MiB │
│ Not Installed │ Cloud Run Proxy │ cloud-run-proxy │ 9.0 MiB │
│ Not Installed │ Cloud SQL Proxy │ cloud_sql_proxy │ 7.8 MiB │
│ Not Installed │ Cloud Spanner Emulator │ cloud-spanner-emulator │ 28.6 MiB │
│ Not Installed │ Cloud Spanner Migration Tool │ harbourbridge │ 16.4 MiB │
│ Not Installed │ Google Container Registry's Docker credential helper │ docker-credential-gcr │ 1.8 MiB │
│ Not Installed │ Kustomize │ kustomize │ 4.3 MiB │
│ Not Installed │ Log Streaming │ log-streaming │ 13.9 MiB │
│ Not Installed │ Minikube │ minikube │ 31.5 MiB │
│ Not Installed │ Nomos CLI │ nomos │ 25.0 MiB │
│ Not Installed │ On-Demand Scanning API extraction helper │ local-extract │ 13.4 MiB │
│ Not Installed │ Skaffold │ skaffold │ 19.7 MiB │
│ Not Installed │ Terraform Tools │ terraform-tools │ 53.3 MiB │
│ Not Installed │ anthos-auth │ anthos-auth │ 20.4 MiB │
│ Not Installed │ config-connector │ config-connector │ 56.7 MiB │
│ Not Installed │ gcloud Alpha Commands │ alpha │ < 1 MiB │
│ Not Installed │ gcloud Beta Commands │ beta │ < 1 MiB │
│ Not Installed │ gcloud app Java Extensions │ app-engine-java │ 64.5 MiB │
│ Not Installed │ gcloud app Python Extensions │ app-engine-python │ 8.6 MiB │
│ Not Installed │ gcloud app Python Extensions (Extra Libraries) │ app-engine-python-extras │ 26.4 MiB │
│ Not Installed │ gke-gcloud-auth-plugin │ gke-gcloud-auth-plugin │ 7.6 MiB │
│ Not Installed │ kpt │ kpt │

In [ ]:
!mkdir profile2
!./google-cloud-sdk/bin/gsutil ls "gs://cvs-synthetic/dipali-test/profile2/"
dbutils.fs.cp("file:///databricks/driver/profile2/", "dbfs:/tmp/profile2", recurse=True)
dbutils.fs.ls("/tmp/data")

gs://cvs-synthetic/dipali-test/profile2/
gs://cvs-synthetic/dipali-test/profile2/cvs_final-profile_1663618982.parquet
Out[3]: [FileInfo(path='dbfs:/tmp/data/000000000000.parquet', name='000000000000.parquet', size=39723836, modificationTime=1663667225000),
 FileInfo(path='dbfs:/tmp/data/000000000001.parquet', name='000000000001.parquet', size=39356828, modificationTime=1663667239000),
 FileInfo(path='dbfs:/tmp/data/000000000002.parquet', name='000000000002.parquet', size=39362872, modificationTime=1663667253000),
 FileInfo(path='dbfs:/tmp/data/000000000003.parquet', name='000000000003.parquet', size=39421412, modificationTime=1663667232000),
 FileInfo(path='dbfs:/tmp/data/000000000004.parquet', name='000000000004.parquet', size=39415391, modificationTime=1663667237000),
 FileInfo(path='dbfs:/tmp/data/000000000005.parquet', name='000000000005.parquet', size=39435480, modificationTime=1663667268000),
 FileInfo(path='dbfs:/tmp/data/000000000006.parquet', name='000000000006.parquet', size=39418602, modificationTime=1663667244000),
 FileInfo(path='dbfs:/tmp/data/000000000007.parquet', name='000000000007.parquet', size=39241704, modificationTime=1663667242000),
 FileInfo(path='dbfs:/tmp/data/000000000008.parquet', name='000000000008.parquet', size=39577482, modificationTime=1663667255000),
 FileInfo(path='dbfs:/tmp/data/000000000009.parquet', name='000000000009.parquet', size=39152124, modificationTime=1663667227000),
 FileInfo(path='dbfs:/tmp/data/000000000010.parquet', name='000000000010.parquet', size=39520088, modificationTime=1663667262000),
 FileInfo(path='dbfs:/tmp/data/000000000011.parquet', name='000000000011.parquet', size=39376551, modificationTime=1663667236000),
 FileInfo(path='dbfs:/tmp/data/000000000012.parquet', name='000000000012.parquet', size=39337253, modificationTime=1663667247000),
 FileInfo(path='dbfs:/tmp/data/000000000013.parquet', name='000000000013.parquet', size=39275527, modificationTime=1663667243000),
 FileInfo(path='dbfs:/tmp/data/000000000014.parquet', name='000000000014.parquet', size=39428607, modificationTime=1663667256000),
 FileInfo(path='dbfs:/tmp/data/000000000015.parquet', name='000000000015.parquet', size=39266219, modificationTime=1663667235000),
 FileInfo(path='dbfs:/tmp/data/000000000016.parquet', name='000000000016.parquet', size=39342315, modificationTime=1663667231000),
 FileInfo(path='dbfs:/tmp/data/000000000017.parquet', name='000000000017.parquet', size=39458295, modificationTime=1663667240000),
 FileInfo(path='dbfs:/tmp/data/000000000018.parquet', name='000000000018.parquet', size=39377592, modificationTime=1663667247000),
 FileInfo(path='dbfs:/tmp/data/000000000019.parquet', name='000000000019.parquet', size=39423139, modificationTime=1663667249000),
 FileInfo(path='dbfs:/tmp/data/000000000020.parquet', name='000000000020.parquet', size=39349352, modificationTime=1663667238000),
 FileInfo(path='dbfs:/tmp/data/000000000021.parquet', name='000000000021.parquet', size=39388104, modificationTime=1663667223000),
 FileInfo(path='dbfs:/tmp/data/000000000022.parquet', name='000000000022.parquet', size=39499665, modificationTime=1663667239000),
 FileInfo(path='dbfs:/tmp/data/000000000023.parquet', name='000000000023.parquet', size=39363011, modificationTime=1663667260000),
 FileInfo(path='dbfs:/tmp/data/000000000024.parquet', name='000000000024.parquet', size=39358701, modificationTime=1663667249000),
 FileInfo(path='dbfs:/tmp/data/000000000025.parquet', name='000000000025.parquet', size=39442894, modificationTime=1663667270000),
 FileInfo(path='dbfs:/tmp/data/000000000026.parquet', name='000000000026.parquet', size=39462973, modificationTime=1663667246000),
 FileInfo(path='dbfs:/tmp/data/000000000027.parquet', name='000000000027.parquet', size=39364198, modificationTime=1663667243000),
 FileInfo(path='dbfs:/tmp/data/000000000028.parquet', name='000000000028.parquet', size=39484345, modificationTime=1663667233000),
 FileInfo(path='dbfs:/tmp/data/000000000029.parquet', name='000000000029.

In [ ]:
# Step 0: Read cvs data
cvs = spark.read.format("parquet") \
    .load("/tmp/data")

In [ ]:
# Step 1: Write a Parquet based table using cvs data
cvs.write.format("parquet").mode("overwrite").save("/tmp/cvs_parquet")

Once step 1 completes, the "cvs" table contains the Data.

Next in Step 2, we run a query that get the record where `MBR_ACCT_GID == "MBR5420929`

In [ ]:
# Step 2: Run a query
 
cvs_parquet = spark.read.format("parquet").load("/tmp/cvs_parquet")
 
display(cvs_parquet.filter(cvs_parquet.MBR_ACCT_GID == "MBR5420929"))

Once step 2 completes, you can observe the latency with the standard "cvs_parquet" table.

In step 3 and step 4, we do the same with a Databricks Delta table. This time, before running the query, we run the OPTIMIZE command with ZORDER to ensure data is optimized for faster retrieval.

In [ ]:
# Step 3: Write a Databricks Delta based table using flights data

cvs.write.format("delta").mode("overwrite").save("/tmp/cvs_delta")

In [ ]:
# Step 3 Continued: OPTIMIZE the Databricks Delta table

spark.conf.set("spark.databricks.delta.optimize.maxFileSize", 128000000)

display(spark.sql("DROP TABLE  IF EXISTS cvs"))
 
display(spark.sql("CREATE TABLE cvs USING DELTA LOCATION '/tmp/cvs_delta'"))
                  
display(spark.sql("OPTIMIZE cvs ZORDER BY (MBR_ACCT_GID)"))

path,metrics
dbfs:/tmp/cvs_delta,"List(27, 34, List(112499913, 144920035, 1.2917650481481482E8, 27, 3487765630), List(35044633, 105673778, 1.0323891188235295E8, 34, 3510123004), 0, List(minCubeSize(107374182400), List(0, 0), List(34, 3510123004), 0, List(34, 3510123004), 1, null), 1, 34, 0, false)"


In [ ]:
# Step 4 : Rerun the query from Step 2 and observe the latency

cvs_delta = spark.read.format("delta").load("/tmp/cvs_delta")

MBR_ACCT_GID,CASES,DRUGS,COUNT_OF_YEAR,MTH_ID,SBSCR_LVL1_ACCT_ID,EPH_LINK_ID,LVL3_ACCT_GID,SBSCR_ACCT_ID,MBR_ACCT_ID,QL_BNFCY_ID,SEG_TYP_DESC,PEER_DESC,MEDD_IND,ELIGIBILITY,NEW_MBR,NEW_PLAN,MULTI_CVRG_IND,MAIL_STATUS,MAIL_STATUS_EPH,TXT_OPT_IN,REGISTERED,APPUSE_IND,MBRPREF_CALL,MBRPREF_EMAIL,MBRPREF_TXT,DIGITAL_STUS,DGTL_ENGMNT_IND,DGTL_RCH_IND,CUSTM_INTGRTN_CLNT_IND,MAIL_UTILIZER,MAINT_IND,MAINT_IND_EPH,RFM_CLM_CNT,SPCLT_CLM_CNT,MAINT_CLM_CNT,RETAIL_CVS_CLM_CNT,RETAIL_NCVS_CLM_CNT,MAIL_CVS_CLM_CNT,MAIL_NCVS_CLM_CNT,TOT_CLM_CNT,RFM_RX_ALCTD_CNT,MAIL_RX_ALCTD_CNT,SPCLT_RX_ALCTD_CNT,MAINT_RX_ALCTD_CNT,TOT_RX_ALCTD_CNT,DRUG_CVRG_AND_COST_CALL_CNT,ORDER_PLACEMNT_CALL_CNT,ORDER_STUS_CALL_CNT,PRIOR_AUTH_CALL_CNT,MBR_ACCT_MAINTENANCE_CALL_CNT,RX_STATUS_CALL_CNT,BILLING_CALL_CNT,PLN_BNFT_OVERRIDE_CALL_CNT,WEBSITE_DETAILS_CALL_CNT,PHMCY_ISSUES_CALL_CNT,RX_HIST_CALL_CNT,RX_DTLS_CALL_CNT,MEDD_GRVNCE_APPLS_CALL_CNT,OTHER_CALL_CNT,TOT_CALL_CNT,TOT_CALL_CNT_NEW,COPAY_DED_CHG_IND,DRG_TIER_FRMLY_CHG_IND,PA_QL_STEP_THRPY_REJ_IND,MAIL_REJ_CNT,SPCLT_REJ_CNT,MAINT_REJ_CNT,TOT_REJ_CNT,AGE_GRP,INCLD_VALID_HH_LVL1_REC,HH_LVL1_EPH_CNT,HH_LVL1_LST_NM_CNT,SEGMENT,SEGMENT_V1,MEDD_CLM_IND,DRUG_PROD_GID,CLM_EVNT_GID,CLM_STUS_CD,CLM_NBR,PHMCY_PTY_GID,ADJD_BRND_GNRC_CD,FRMLY_ID,FILL_DT,CLM_SEQ_NBR,FRMLY_PDL_CD,LICS_LVL_CD,MAINT_CHOICE_ELIG_CNT,RX_WRTN_DT,RX_ORGN_CD,EVNT_DT,NCPDP_RJCT2_CD,MBR_PRIOR_ATHZN_NBR,DAY_SPLY_QTY,NCPDP_RJCT3_CD,OVRD_RJCT_PRIOR_ATHZN_CD,INDVL_DEDTBL_AMT,NCPDP_RJCT1_CD,ADJD_MAINT_DRUG_IND,DRUG_ID,DRUG_LBL_NM,NCPDP_PRVDR_ID,NPI_ID,PHMCY_NM,PHMCY_NTWRK_MAIL_RTL_CD,PRIOR_ATHZN_MAX_DLY_DSG,RX_NBR,ADDR_LINE2,ADDR_ZIP5_CD,BNFT_PLAN_RESET_DT,CURR_ELIG_IND,DTH_DT,EPH_ID,GNDR_CD,LANG_CD,LVL1_ACCT_GID,MBR_BRTH_DT,MBR_FRST_NM,MBR_LAST_NM,PHONE_NBR,PTNT_GID,CDM_GBL_EMAIL_EVAL_CD,CDM_GBL_SMS_EVAL_CD,CDM_GBL_IVR_EVAL_CD,MBR_EMAIL_TYP3_TXT,MBR_EMAIL_TYP4_TXT,MBR_EMAIL_TYP5_TXT,MBR_EMAIL_TYP6_TXT,PAYLOAD_EMAIL_TXT,PAYLOAD_SMS_NBR,PAYLOAD_IVR_NBR,VALID_EMAIL_TYP3_IND,VALID_EMAIL_TYP4_IND,VALID_EMAIL_TYP5_IND,VALID_EMAIL_TYP6_IND,VALID_PAYLOAD_EMAIL_IND,VALID_PAYLOAD_SMS_IND,VALID_PAYLOAD_IVR_IND,CDM_GBL_EMAIL_EVAL_IND,CDM_GBL_SMS_EVAL_IND,CDM_GBL_IVR_EVAL_IND,PGM_ID,CMNCT_ID,CHNL_PRTY_EMAIL_CD,CHNL_PRTY_SMS_CD,CHNL_PRTY_IVR_CD,CHNL_DFLT_CD,REC_EFF_DT,REC_EXPRN_DT,CURR_IND
MBR5420929,33,36,1,202001,16253495154866,73671474343,20562309248589,203881042,56080,QL5420929,MEDICARE,MAPD,MEDD,ELIGIBLE,NOT ELIGIBLE,NEW PLAN,0,NA,MAIL CONTINUE,N,0,1,0,1,0,ACTIVE,1,0,2,N,NA,NA,4,0,7,15,4,0,1,19,4,14,0,16,14,1,1,1,4,1,1,1,0,1,1,0,0,1,1,4,7,0,0,0,0,0,13,40,75-84,1,1,1,RETAIL MEDD,Mail Continuer w/ less than 5 Maintenance Claims,N,64600,26086812331,V,199951717818513,103479,GNRC,juCAH,1958-09-25T07:36:10Z,994,PREF,3,1,1995-03-26T05:00:44Z,EL,1906-01-19T06:07:22Z,569,1933798505,7,,GJQqTVPh,2,76,N,7487597570,PREDNISONE TAB 10MG,1307594,1710056445,jjIouiU,O,8,VthaBt,APT 703,,1968-01-10T02:18:48Z,N,1900-02-16T11:24:19Z,2,M,ENGL,3,2007-08-08T13:10:50Z,Bianka,Mayert,5034799966,2,2,2,-1,JQHzmAjHCG,hOED,OiFWARroxd,dLGTFGfOo,maceyherman@volkman.biz,8949826776,748944163,Y,N,N,Y,N,N,N,Y,N,Y,108,2624,1,2,3,E,1976-04-26T01:47:05Z,1963-06-27T08:06:15Z,N


In [ ]:
display(cvs_parquet.filter(cvs_parquet.MBR_ACCT_GID == "MBR5420929"))

MBR_ACCT_GID,CASES,DRUGS,COUNT_OF_YEAR,MTH_ID,SBSCR_LVL1_ACCT_ID,EPH_LINK_ID,LVL3_ACCT_GID,SBSCR_ACCT_ID,MBR_ACCT_ID,QL_BNFCY_ID,SEG_TYP_DESC,PEER_DESC,MEDD_IND,ELIGIBILITY,NEW_MBR,NEW_PLAN,MULTI_CVRG_IND,MAIL_STATUS,MAIL_STATUS_EPH,TXT_OPT_IN,REGISTERED,APPUSE_IND,MBRPREF_CALL,MBRPREF_EMAIL,MBRPREF_TXT,DIGITAL_STUS,DGTL_ENGMNT_IND,DGTL_RCH_IND,CUSTM_INTGRTN_CLNT_IND,MAIL_UTILIZER,MAINT_IND,MAINT_IND_EPH,RFM_CLM_CNT,SPCLT_CLM_CNT,MAINT_CLM_CNT,RETAIL_CVS_CLM_CNT,RETAIL_NCVS_CLM_CNT,MAIL_CVS_CLM_CNT,MAIL_NCVS_CLM_CNT,TOT_CLM_CNT,RFM_RX_ALCTD_CNT,MAIL_RX_ALCTD_CNT,SPCLT_RX_ALCTD_CNT,MAINT_RX_ALCTD_CNT,TOT_RX_ALCTD_CNT,DRUG_CVRG_AND_COST_CALL_CNT,ORDER_PLACEMNT_CALL_CNT,ORDER_STUS_CALL_CNT,PRIOR_AUTH_CALL_CNT,MBR_ACCT_MAINTENANCE_CALL_CNT,RX_STATUS_CALL_CNT,BILLING_CALL_CNT,PLN_BNFT_OVERRIDE_CALL_CNT,WEBSITE_DETAILS_CALL_CNT,PHMCY_ISSUES_CALL_CNT,RX_HIST_CALL_CNT,RX_DTLS_CALL_CNT,MEDD_GRVNCE_APPLS_CALL_CNT,OTHER_CALL_CNT,TOT_CALL_CNT,TOT_CALL_CNT_NEW,COPAY_DED_CHG_IND,DRG_TIER_FRMLY_CHG_IND,PA_QL_STEP_THRPY_REJ_IND,MAIL_REJ_CNT,SPCLT_REJ_CNT,MAINT_REJ_CNT,TOT_REJ_CNT,AGE_GRP,INCLD_VALID_HH_LVL1_REC,HH_LVL1_EPH_CNT,HH_LVL1_LST_NM_CNT,SEGMENT,SEGMENT_V1,MEDD_CLM_IND,DRUG_PROD_GID,CLM_EVNT_GID,CLM_STUS_CD,CLM_NBR,PHMCY_PTY_GID,ADJD_BRND_GNRC_CD,FRMLY_ID,FILL_DT,CLM_SEQ_NBR,FRMLY_PDL_CD,LICS_LVL_CD,MAINT_CHOICE_ELIG_CNT,RX_WRTN_DT,RX_ORGN_CD,EVNT_DT,NCPDP_RJCT2_CD,MBR_PRIOR_ATHZN_NBR,DAY_SPLY_QTY,NCPDP_RJCT3_CD,OVRD_RJCT_PRIOR_ATHZN_CD,INDVL_DEDTBL_AMT,NCPDP_RJCT1_CD,ADJD_MAINT_DRUG_IND,DRUG_ID,DRUG_LBL_NM,NCPDP_PRVDR_ID,NPI_ID,PHMCY_NM,PHMCY_NTWRK_MAIL_RTL_CD,PRIOR_ATHZN_MAX_DLY_DSG,RX_NBR,ADDR_LINE2,ADDR_ZIP5_CD,BNFT_PLAN_RESET_DT,CURR_ELIG_IND,DTH_DT,EPH_ID,GNDR_CD,LANG_CD,LVL1_ACCT_GID,MBR_BRTH_DT,MBR_FRST_NM,MBR_LAST_NM,PHONE_NBR,PTNT_GID,CDM_GBL_EMAIL_EVAL_CD,CDM_GBL_SMS_EVAL_CD,CDM_GBL_IVR_EVAL_CD,MBR_EMAIL_TYP3_TXT,MBR_EMAIL_TYP4_TXT,MBR_EMAIL_TYP5_TXT,MBR_EMAIL_TYP6_TXT,PAYLOAD_EMAIL_TXT,PAYLOAD_SMS_NBR,PAYLOAD_IVR_NBR,VALID_EMAIL_TYP3_IND,VALID_EMAIL_TYP4_IND,VALID_EMAIL_TYP5_IND,VALID_EMAIL_TYP6_IND,VALID_PAYLOAD_EMAIL_IND,VALID_PAYLOAD_SMS_IND,VALID_PAYLOAD_IVR_IND,CDM_GBL_EMAIL_EVAL_IND,CDM_GBL_SMS_EVAL_IND,CDM_GBL_IVR_EVAL_IND,PGM_ID,CMNCT_ID,CHNL_PRTY_EMAIL_CD,CHNL_PRTY_SMS_CD,CHNL_PRTY_IVR_CD,CHNL_DFLT_CD,REC_EFF_DT,REC_EXPRN_DT,CURR_IND
MBR5420929,33,36,1,202001,16253495154866,73671474343,20562309248589,203881042,56080,QL5420929,MEDICARE,MAPD,MEDD,ELIGIBLE,NOT ELIGIBLE,NEW PLAN,0,NA,MAIL CONTINUE,N,0,1,0,1,0,ACTIVE,1,0,2,N,NA,NA,4,0,7,15,4,0,1,19,4,14,0,16,14,1,1,1,4,1,1,1,0,1,1,0,0,1,1,4,7,0,0,0,0,0,13,40,75-84,1,1,1,RETAIL MEDD,Mail Continuer w/ less than 5 Maintenance Claims,N,64600,26086812331,V,199951717818513,103479,GNRC,juCAH,1958-09-25T07:36:10Z,994,PREF,3,1,1995-03-26T05:00:44Z,EL,1906-01-19T06:07:22Z,569,1933798505,7,,GJQqTVPh,2,76,N,7487597570,PREDNISONE TAB 10MG,1307594,1710056445,jjIouiU,O,8,VthaBt,APT 703,,1968-01-10T02:18:48Z,N,1900-02-16T11:24:19Z,2,M,ENGL,3,2007-08-08T13:10:50Z,Bianka,Mayert,5034799966,2,2,2,-1,JQHzmAjHCG,hOED,OiFWARroxd,dLGTFGfOo,maceyherman@volkman.biz,8949826776,748944163,Y,N,N,Y,N,N,N,Y,N,Y,108,2624,1,2,3,E,1976-04-26T01:47:05Z,1963-06-27T08:06:15Z,N


In [ ]:
display(cvs_parquet.filter(cvs_parquet.MBR_ACCT_GID == "MBR10090397"))

MBR_ACCT_GID,CASES,DRUGS,COUNT_OF_YEAR,MTH_ID,SBSCR_LVL1_ACCT_ID,EPH_LINK_ID,LVL3_ACCT_GID,SBSCR_ACCT_ID,MBR_ACCT_ID,QL_BNFCY_ID,SEG_TYP_DESC,PEER_DESC,MEDD_IND,ELIGIBILITY,NEW_MBR,NEW_PLAN,MULTI_CVRG_IND,MAIL_STATUS,MAIL_STATUS_EPH,TXT_OPT_IN,REGISTERED,APPUSE_IND,MBRPREF_CALL,MBRPREF_EMAIL,MBRPREF_TXT,DIGITAL_STUS,DGTL_ENGMNT_IND,DGTL_RCH_IND,CUSTM_INTGRTN_CLNT_IND,MAIL_UTILIZER,MAINT_IND,MAINT_IND_EPH,RFM_CLM_CNT,SPCLT_CLM_CNT,MAINT_CLM_CNT,RETAIL_CVS_CLM_CNT,RETAIL_NCVS_CLM_CNT,MAIL_CVS_CLM_CNT,MAIL_NCVS_CLM_CNT,TOT_CLM_CNT,RFM_RX_ALCTD_CNT,MAIL_RX_ALCTD_CNT,SPCLT_RX_ALCTD_CNT,MAINT_RX_ALCTD_CNT,TOT_RX_ALCTD_CNT,DRUG_CVRG_AND_COST_CALL_CNT,ORDER_PLACEMNT_CALL_CNT,ORDER_STUS_CALL_CNT,PRIOR_AUTH_CALL_CNT,MBR_ACCT_MAINTENANCE_CALL_CNT,RX_STATUS_CALL_CNT,BILLING_CALL_CNT,PLN_BNFT_OVERRIDE_CALL_CNT,WEBSITE_DETAILS_CALL_CNT,PHMCY_ISSUES_CALL_CNT,RX_HIST_CALL_CNT,RX_DTLS_CALL_CNT,MEDD_GRVNCE_APPLS_CALL_CNT,OTHER_CALL_CNT,TOT_CALL_CNT,TOT_CALL_CNT_NEW,COPAY_DED_CHG_IND,DRG_TIER_FRMLY_CHG_IND,PA_QL_STEP_THRPY_REJ_IND,MAIL_REJ_CNT,SPCLT_REJ_CNT,MAINT_REJ_CNT,TOT_REJ_CNT,AGE_GRP,INCLD_VALID_HH_LVL1_REC,HH_LVL1_EPH_CNT,HH_LVL1_LST_NM_CNT,SEGMENT,SEGMENT_V1,MEDD_CLM_IND,DRUG_PROD_GID,CLM_EVNT_GID,CLM_STUS_CD,CLM_NBR,PHMCY_PTY_GID,ADJD_BRND_GNRC_CD,FRMLY_ID,FILL_DT,CLM_SEQ_NBR,FRMLY_PDL_CD,LICS_LVL_CD,MAINT_CHOICE_ELIG_CNT,RX_WRTN_DT,RX_ORGN_CD,EVNT_DT,NCPDP_RJCT2_CD,MBR_PRIOR_ATHZN_NBR,DAY_SPLY_QTY,NCPDP_RJCT3_CD,OVRD_RJCT_PRIOR_ATHZN_CD,INDVL_DEDTBL_AMT,NCPDP_RJCT1_CD,ADJD_MAINT_DRUG_IND,DRUG_ID,DRUG_LBL_NM,NCPDP_PRVDR_ID,NPI_ID,PHMCY_NM,PHMCY_NTWRK_MAIL_RTL_CD,PRIOR_ATHZN_MAX_DLY_DSG,RX_NBR,ADDR_LINE2,ADDR_ZIP5_CD,BNFT_PLAN_RESET_DT,CURR_ELIG_IND,DTH_DT,EPH_ID,GNDR_CD,LANG_CD,LVL1_ACCT_GID,MBR_BRTH_DT,MBR_FRST_NM,MBR_LAST_NM,PHONE_NBR,PTNT_GID,CDM_GBL_EMAIL_EVAL_CD,CDM_GBL_SMS_EVAL_CD,CDM_GBL_IVR_EVAL_CD,MBR_EMAIL_TYP3_TXT,MBR_EMAIL_TYP4_TXT,MBR_EMAIL_TYP5_TXT,MBR_EMAIL_TYP6_TXT,PAYLOAD_EMAIL_TXT,PAYLOAD_SMS_NBR,PAYLOAD_IVR_NBR,VALID_EMAIL_TYP3_IND,VALID_EMAIL_TYP4_IND,VALID_EMAIL_TYP5_IND,VALID_EMAIL_TYP6_IND,VALID_PAYLOAD_EMAIL_IND,VALID_PAYLOAD_SMS_IND,VALID_PAYLOAD_IVR_IND,CDM_GBL_EMAIL_EVAL_IND,CDM_GBL_SMS_EVAL_IND,CDM_GBL_IVR_EVAL_IND,PGM_ID,CMNCT_ID,CHNL_PRTY_EMAIL_CD,CHNL_PRTY_SMS_CD,CHNL_PRTY_IVR_CD,CHNL_DFLT_CD,REC_EFF_DT,REC_EXPRN_DT,CURR_IND
MBR10090397,39,5,2,201807,16790312447778,72117776308,21003004336535,214872757,33590,QL10090397,MEDICARE,PDP,MEDD,ELIGIBLE,EXISTING MEMBER,EXISTING PLAN,1,MAIL CONTINUE,NON MAIL,N,1,1,1,0,1,UNENGAGED,0,0,3,N,NA,MAINT,1,1,28,4,29,4,1,18,3,1,5,30,32,0,1,1,2,0,0,0,0,1,0,0,1,1,0,0,3,1,0,1,1,1,4,15,65-74,1,1,1,MAIL MEDD,Mail Continuer w/ 5 or more Maintenance Claims,Y,309803,20622374622,V,196785051030733,103479,BRND,RczcCPynS,1979-11-26T13:28:21Z,993,N-PREF,3,1,1990-07-24T15:18:55Z,EL,1926-01-10T17:10:18Z,569,1933798465,5,,lyoO,2,,Y,33165902052,SUMATRIPTAN TAB 100MG,1307594,1710056445,nMkSRflsTi,O,4,IUcsgadZ,APT 732,,1992-10-19T06:42:39Z,N,1972-05-27T01:02:40Z,6,M,ENGL,8,1954-07-03T12:00:30Z,Monica,Hahn,4114450066,1,1,2,1,ZvYPodKDje,LImGm,yXJUbWTgbV,gQrNwOOUAa,mariahhermiston@friesen.io,9186166478,723313534,N,N,N,Y,Y,N,Y,N,Y,Y,108,2651,1,2,3,E,1934-02-12T19:55:25Z,2015-10-05T05:03:34Z,N


The query over the Databricks Delta table runs much faster after OPTIMIZE is run. How much faster the query runs can depend on the configuration of the cluster you are running on, however should be 5-10X faster compared to the standard table

In [ ]:
# Step 5 : Rerun the query from Step 2 and observe the latency after caching the table

cvs_delta_cache = spark.read.format("delta").load("/tmp/cvs_delta")

In [ ]:
cvs_delta_cache.cache()
cvs_delta_cache.count()

Out[13]: 10000000

In [ ]:
display(cvs_parquet.filter(cvs_parquet.MBR_ACCT_GID == "MBR5420929"))

MBR_ACCT_GID,CASES,DRUGS,COUNT_OF_YEAR,MTH_ID,SBSCR_LVL1_ACCT_ID,EPH_LINK_ID,LVL3_ACCT_GID,SBSCR_ACCT_ID,MBR_ACCT_ID,QL_BNFCY_ID,SEG_TYP_DESC,PEER_DESC,MEDD_IND,ELIGIBILITY,NEW_MBR,NEW_PLAN,MULTI_CVRG_IND,MAIL_STATUS,MAIL_STATUS_EPH,TXT_OPT_IN,REGISTERED,APPUSE_IND,MBRPREF_CALL,MBRPREF_EMAIL,MBRPREF_TXT,DIGITAL_STUS,DGTL_ENGMNT_IND,DGTL_RCH_IND,CUSTM_INTGRTN_CLNT_IND,MAIL_UTILIZER,MAINT_IND,MAINT_IND_EPH,RFM_CLM_CNT,SPCLT_CLM_CNT,MAINT_CLM_CNT,RETAIL_CVS_CLM_CNT,RETAIL_NCVS_CLM_CNT,MAIL_CVS_CLM_CNT,MAIL_NCVS_CLM_CNT,TOT_CLM_CNT,RFM_RX_ALCTD_CNT,MAIL_RX_ALCTD_CNT,SPCLT_RX_ALCTD_CNT,MAINT_RX_ALCTD_CNT,TOT_RX_ALCTD_CNT,DRUG_CVRG_AND_COST_CALL_CNT,ORDER_PLACEMNT_CALL_CNT,ORDER_STUS_CALL_CNT,PRIOR_AUTH_CALL_CNT,MBR_ACCT_MAINTENANCE_CALL_CNT,RX_STATUS_CALL_CNT,BILLING_CALL_CNT,PLN_BNFT_OVERRIDE_CALL_CNT,WEBSITE_DETAILS_CALL_CNT,PHMCY_ISSUES_CALL_CNT,RX_HIST_CALL_CNT,RX_DTLS_CALL_CNT,MEDD_GRVNCE_APPLS_CALL_CNT,OTHER_CALL_CNT,TOT_CALL_CNT,TOT_CALL_CNT_NEW,COPAY_DED_CHG_IND,DRG_TIER_FRMLY_CHG_IND,PA_QL_STEP_THRPY_REJ_IND,MAIL_REJ_CNT,SPCLT_REJ_CNT,MAINT_REJ_CNT,TOT_REJ_CNT,AGE_GRP,INCLD_VALID_HH_LVL1_REC,HH_LVL1_EPH_CNT,HH_LVL1_LST_NM_CNT,SEGMENT,SEGMENT_V1,MEDD_CLM_IND,DRUG_PROD_GID,CLM_EVNT_GID,CLM_STUS_CD,CLM_NBR,PHMCY_PTY_GID,ADJD_BRND_GNRC_CD,FRMLY_ID,FILL_DT,CLM_SEQ_NBR,FRMLY_PDL_CD,LICS_LVL_CD,MAINT_CHOICE_ELIG_CNT,RX_WRTN_DT,RX_ORGN_CD,EVNT_DT,NCPDP_RJCT2_CD,MBR_PRIOR_ATHZN_NBR,DAY_SPLY_QTY,NCPDP_RJCT3_CD,OVRD_RJCT_PRIOR_ATHZN_CD,INDVL_DEDTBL_AMT,NCPDP_RJCT1_CD,ADJD_MAINT_DRUG_IND,DRUG_ID,DRUG_LBL_NM,NCPDP_PRVDR_ID,NPI_ID,PHMCY_NM,PHMCY_NTWRK_MAIL_RTL_CD,PRIOR_ATHZN_MAX_DLY_DSG,RX_NBR,ADDR_LINE2,ADDR_ZIP5_CD,BNFT_PLAN_RESET_DT,CURR_ELIG_IND,DTH_DT,EPH_ID,GNDR_CD,LANG_CD,LVL1_ACCT_GID,MBR_BRTH_DT,MBR_FRST_NM,MBR_LAST_NM,PHONE_NBR,PTNT_GID,CDM_GBL_EMAIL_EVAL_CD,CDM_GBL_SMS_EVAL_CD,CDM_GBL_IVR_EVAL_CD,MBR_EMAIL_TYP3_TXT,MBR_EMAIL_TYP4_TXT,MBR_EMAIL_TYP5_TXT,MBR_EMAIL_TYP6_TXT,PAYLOAD_EMAIL_TXT,PAYLOAD_SMS_NBR,PAYLOAD_IVR_NBR,VALID_EMAIL_TYP3_IND,VALID_EMAIL_TYP4_IND,VALID_EMAIL_TYP5_IND,VALID_EMAIL_TYP6_IND,VALID_PAYLOAD_EMAIL_IND,VALID_PAYLOAD_SMS_IND,VALID_PAYLOAD_IVR_IND,CDM_GBL_EMAIL_EVAL_IND,CDM_GBL_SMS_EVAL_IND,CDM_GBL_IVR_EVAL_IND,PGM_ID,CMNCT_ID,CHNL_PRTY_EMAIL_CD,CHNL_PRTY_SMS_CD,CHNL_PRTY_IVR_CD,CHNL_DFLT_CD,REC_EFF_DT,REC_EXPRN_DT,CURR_IND
MBR5420929,33,36,1,202001,16253495154866,73671474343,20562309248589,203881042,56080,QL5420929,MEDICARE,MAPD,MEDD,ELIGIBLE,NOT ELIGIBLE,NEW PLAN,0,NA,MAIL CONTINUE,N,0,1,0,1,0,ACTIVE,1,0,2,N,NA,NA,4,0,7,15,4,0,1,19,4,14,0,16,14,1,1,1,4,1,1,1,0,1,1,0,0,1,1,4,7,0,0,0,0,0,13,40,75-84,1,1,1,RETAIL MEDD,Mail Continuer w/ less than 5 Maintenance Claims,N,64600,26086812331,V,199951717818513,103479,GNRC,juCAH,1958-09-25T07:36:10Z,994,PREF,3,1,1995-03-26T05:00:44Z,EL,1906-01-19T06:07:22Z,569,1933798505,7,,GJQqTVPh,2,76,N,7487597570,PREDNISONE TAB 10MG,1307594,1710056445,jjIouiU,O,8,VthaBt,APT 703,,1968-01-10T02:18:48Z,N,1900-02-16T11:24:19Z,2,M,ENGL,3,2007-08-08T13:10:50Z,Bianka,Mayert,5034799966,2,2,2,-1,JQHzmAjHCG,hOED,OiFWARroxd,dLGTFGfOo,maceyherman@volkman.biz,8949826776,748944163,Y,N,N,Y,N,N,N,Y,N,Y,108,2624,1,2,3,E,1976-04-26T01:47:05Z,1963-06-27T08:06:15Z,N


In [ ]:
display(cvs_parquet.filter(cvs_parquet.MBR_ACCT_GID == "MBR10090397"))

MBR_ACCT_GID,CASES,DRUGS,COUNT_OF_YEAR,MTH_ID,SBSCR_LVL1_ACCT_ID,EPH_LINK_ID,LVL3_ACCT_GID,SBSCR_ACCT_ID,MBR_ACCT_ID,QL_BNFCY_ID,SEG_TYP_DESC,PEER_DESC,MEDD_IND,ELIGIBILITY,NEW_MBR,NEW_PLAN,MULTI_CVRG_IND,MAIL_STATUS,MAIL_STATUS_EPH,TXT_OPT_IN,REGISTERED,APPUSE_IND,MBRPREF_CALL,MBRPREF_EMAIL,MBRPREF_TXT,DIGITAL_STUS,DGTL_ENGMNT_IND,DGTL_RCH_IND,CUSTM_INTGRTN_CLNT_IND,MAIL_UTILIZER,MAINT_IND,MAINT_IND_EPH,RFM_CLM_CNT,SPCLT_CLM_CNT,MAINT_CLM_CNT,RETAIL_CVS_CLM_CNT,RETAIL_NCVS_CLM_CNT,MAIL_CVS_CLM_CNT,MAIL_NCVS_CLM_CNT,TOT_CLM_CNT,RFM_RX_ALCTD_CNT,MAIL_RX_ALCTD_CNT,SPCLT_RX_ALCTD_CNT,MAINT_RX_ALCTD_CNT,TOT_RX_ALCTD_CNT,DRUG_CVRG_AND_COST_CALL_CNT,ORDER_PLACEMNT_CALL_CNT,ORDER_STUS_CALL_CNT,PRIOR_AUTH_CALL_CNT,MBR_ACCT_MAINTENANCE_CALL_CNT,RX_STATUS_CALL_CNT,BILLING_CALL_CNT,PLN_BNFT_OVERRIDE_CALL_CNT,WEBSITE_DETAILS_CALL_CNT,PHMCY_ISSUES_CALL_CNT,RX_HIST_CALL_CNT,RX_DTLS_CALL_CNT,MEDD_GRVNCE_APPLS_CALL_CNT,OTHER_CALL_CNT,TOT_CALL_CNT,TOT_CALL_CNT_NEW,COPAY_DED_CHG_IND,DRG_TIER_FRMLY_CHG_IND,PA_QL_STEP_THRPY_REJ_IND,MAIL_REJ_CNT,SPCLT_REJ_CNT,MAINT_REJ_CNT,TOT_REJ_CNT,AGE_GRP,INCLD_VALID_HH_LVL1_REC,HH_LVL1_EPH_CNT,HH_LVL1_LST_NM_CNT,SEGMENT,SEGMENT_V1,MEDD_CLM_IND,DRUG_PROD_GID,CLM_EVNT_GID,CLM_STUS_CD,CLM_NBR,PHMCY_PTY_GID,ADJD_BRND_GNRC_CD,FRMLY_ID,FILL_DT,CLM_SEQ_NBR,FRMLY_PDL_CD,LICS_LVL_CD,MAINT_CHOICE_ELIG_CNT,RX_WRTN_DT,RX_ORGN_CD,EVNT_DT,NCPDP_RJCT2_CD,MBR_PRIOR_ATHZN_NBR,DAY_SPLY_QTY,NCPDP_RJCT3_CD,OVRD_RJCT_PRIOR_ATHZN_CD,INDVL_DEDTBL_AMT,NCPDP_RJCT1_CD,ADJD_MAINT_DRUG_IND,DRUG_ID,DRUG_LBL_NM,NCPDP_PRVDR_ID,NPI_ID,PHMCY_NM,PHMCY_NTWRK_MAIL_RTL_CD,PRIOR_ATHZN_MAX_DLY_DSG,RX_NBR,ADDR_LINE2,ADDR_ZIP5_CD,BNFT_PLAN_RESET_DT,CURR_ELIG_IND,DTH_DT,EPH_ID,GNDR_CD,LANG_CD,LVL1_ACCT_GID,MBR_BRTH_DT,MBR_FRST_NM,MBR_LAST_NM,PHONE_NBR,PTNT_GID,CDM_GBL_EMAIL_EVAL_CD,CDM_GBL_SMS_EVAL_CD,CDM_GBL_IVR_EVAL_CD,MBR_EMAIL_TYP3_TXT,MBR_EMAIL_TYP4_TXT,MBR_EMAIL_TYP5_TXT,MBR_EMAIL_TYP6_TXT,PAYLOAD_EMAIL_TXT,PAYLOAD_SMS_NBR,PAYLOAD_IVR_NBR,VALID_EMAIL_TYP3_IND,VALID_EMAIL_TYP4_IND,VALID_EMAIL_TYP5_IND,VALID_EMAIL_TYP6_IND,VALID_PAYLOAD_EMAIL_IND,VALID_PAYLOAD_SMS_IND,VALID_PAYLOAD_IVR_IND,CDM_GBL_EMAIL_EVAL_IND,CDM_GBL_SMS_EVAL_IND,CDM_GBL_IVR_EVAL_IND,PGM_ID,CMNCT_ID,CHNL_PRTY_EMAIL_CD,CHNL_PRTY_SMS_CD,CHNL_PRTY_IVR_CD,CHNL_DFLT_CD,REC_EFF_DT,REC_EXPRN_DT,CURR_IND
MBR10090397,39,5,2,201807,16790312447778,72117776308,21003004336535,214872757,33590,QL10090397,MEDICARE,PDP,MEDD,ELIGIBLE,EXISTING MEMBER,EXISTING PLAN,1,MAIL CONTINUE,NON MAIL,N,1,1,1,0,1,UNENGAGED,0,0,3,N,NA,MAINT,1,1,28,4,29,4,1,18,3,1,5,30,32,0,1,1,2,0,0,0,0,1,0,0,1,1,0,0,3,1,0,1,1,1,4,15,65-74,1,1,1,MAIL MEDD,Mail Continuer w/ 5 or more Maintenance Claims,Y,309803,20622374622,V,196785051030733,103479,BRND,RczcCPynS,1979-11-26T13:28:21Z,993,N-PREF,3,1,1990-07-24T15:18:55Z,EL,1926-01-10T17:10:18Z,569,1933798465,5,,lyoO,2,,Y,33165902052,SUMATRIPTAN TAB 100MG,1307594,1710056445,nMkSRflsTi,O,4,IUcsgadZ,APT 732,,1992-10-19T06:42:39Z,N,1972-05-27T01:02:40Z,6,M,ENGL,8,1954-07-03T12:00:30Z,Monica,Hahn,4114450066,1,1,2,1,ZvYPodKDje,LImGm,yXJUbWTgbV,gQrNwOOUAa,mariahhermiston@friesen.io,9186166478,723313534,N,N,N,Y,Y,N,Y,N,Y,Y,108,2651,1,2,3,E,1934-02-12T19:55:25Z,2015-10-05T05:03:34Z,N
